In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import datetime
import pandas as pd
import pandasql as ps
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
agg3 =  pd.read_csv('./mid/agg3.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-23 09:16:45.725226
종료: 2024-02-23 09:16:57.958981


In [5]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)

merged_references_991_to_2028 = pd.read_csv('./reference/merged_references_991_to_2028.txt',  sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-23 09:22:49.812697
종료: 2024-02-23 09:33:29.798942


In [4]:
agg3

,uid,cluster_id,pubyear2
0,WOS:000239031100003,1333,2006
1,WOS:000234737300014,846,2006
2,WOS:000241844400003,997,2006
3,WOS:000417059300001,151,2006
4,WOS:000235792300010,210,2006
...,...,...,...
24339950,WOS:000750113400001,2719,2021
24339951,WOS:000839344100002,1636,2022
24339952,WOS:000798801800001,3584,2021
24339953,WOS:000763296900001,2034,2021


In [6]:
merged_references_991_to_2028

,uid,ref_uid,ref_count
0,WOS:000325157800001,WOS:000169875200049,59
1,WOS:000325157800001,WOS:000187577400045,59
2,WOS:000325157800001,WOS:000180545300005,59
3,WOS:000325157800001,WOS:000188445200197,59
4,WOS:000325157800001,WOS:A1972M852600031,59
...,...,...,...
1037622920,WOS:000799950200001,000799950200001.23,52
1037622921,WOS:000799950200001,000799950200001.22,52
1037622922,WOS:000799950200001,WOS:000316422500018,52
1037622923,WOS:000799950200001,000799950200001.48,52


In [7]:
# 각 컬럼별 null 값의 개수 계산
null_counts000 = merged_references_991_to_2028.isnull().sum()

# 결과 출력
print(null_counts000)

uid              0
ref_uid      69991
ref_count        0
dtype: int64


조인하기

In [8]:
now = datetime.datetime.now() 
print('시작:', now)

third_result4 = pd.merge(merged_references_991_to_2028,agg3, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-23 09:34:52.040620
종료: 2024-02-23 09:37:52.762351


In [9]:
# 각 컬럼별 null 값의 개수 계산
null_counts4 = third_result4.isnull().sum()

# 결과 출력
print(null_counts4)

uid                   0
ref_uid           69991
ref_count             0
cluster_id    332143243
pubyear2      332143243
dtype: int64


In [10]:
now = datetime.datetime.now() 
print('시작:', now)

# 컬럼명 변경
third_result4 = third_result4.rename(columns={'ref_uid': 'refs_uid', 'ref_count': 'refs_n'})


# 컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id','pubyear2', 'refs_uid', 'refs_n']

# 새 순서로 DataFrame을 재구성
third_result4 = third_result4[new_order]

# 정수화
third_result4['cluster_id'] = third_result4['cluster_id'].astype('Int64')
third_result4['pubyear2'] = third_result4['pubyear2'].astype('Int64')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-23 09:38:44.565497
종료: 2024-02-23 09:42:28.214196


In [11]:
third_result4

,uid,cluster_id,pubyear2,refs_uid,refs_n
0,WOS:000325157800001,25,2013,WOS:000169875200049,59
1,WOS:000325157800001,25,2013,WOS:000187577400045,59
2,WOS:000325157800001,25,2013,WOS:000180545300005,59
3,WOS:000325157800001,25,2013,WOS:000188445200197,59
4,WOS:000325157800001,25,2013,WOS:A1972M852600031,59
...,...,...,...,...,...
1037622920,WOS:000799950200001,<NA>,<NA>,000799950200001.23,52
1037622921,WOS:000799950200001,<NA>,<NA>,000799950200001.22,52
1037622922,WOS:000799950200001,<NA>,<NA>,WOS:000316422500018,52
1037622923,WOS:000799950200001,<NA>,<NA>,000799950200001.48,52


ut_to_uid_is_core_set.pkl => P에서 ut를 통해 ut_to_uid(=uid)를 만듦 -> core가 1인 ut_to_uid(uid)만 추출한 set임.
merged_references의 uid가 이 pkl에 있다면(P에 존재하고 CORE가 1임) within_core 가 1, 아님 0으로 값 조정.

In [12]:
import pickle

# pickle 파일로부터 set 데이터 불러오기
with open('./mid/ut_to_uid_is_core_set.pkl', 'rb') as file:
    loaded_is_core_set = pickle.load(file)

#print(loaded_ut_ut_to_uid_is_core_set)
print(f"ut_to_uid 컬럼 데이터의 set 크기: {len(loaded_is_core_set)}")

ut_to_uid 컬럼 데이터의 set 크기: 22379399


refs_uid에 결측값이 존재함. 0과 1을 정수형으로 저장할 수 없음. 따라서 within_core의 데이터타입을 문자열로 지정함. 

이 코드를 통해 각 행에 대해 refs_uid 값이 결측치인 경우 'NaN'을, loaded_is_core_set에 포함되어 있는 경우 '1'을, 그렇지 않은 경우 '0'을 within_core 컬럼에 할당합니다

In [14]:
import numpy as np


now = datetime.datetime.now() 
print('시작:', now)

# third_result2 DataFrame에 within_core 컬럼 추가 및 값 할당, 문자열로 처리
third_result4['within_core'] = third_result4['refs_uid'].apply(lambda x: 'NaN' if pd.isnull(x) else ('1' if x in loaded_is_core_set else '0'))

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-23 09:58:12.133907
종료: 2024-02-23 10:10:46.958020


 MemoryError: Unable to allocate 7.73 GiB for an array with shape (1037622925,) and data type uint64 오류는 파이썬이 요청된 크기의 배열을 메모리에 할당할 수 없을 때 발생합니다. 이 오류는 대규모 데이터를 처리하려고 할 때 특히 빈번하게 발생하며, 시스템의 사용 가능한 RAM이 요구되는 메모리 양보다 적을 때 나타납니다. 

따라서 df를 반으로 쪼개서 작업을 시도하려했으나, 위 코드 다시 돌리니 돌아가서 그대로 사용.

within_core의 데이터 값 조사, 카운트 

In [15]:
now = datetime.datetime.now() 
print('시작:', now)


# within_core 컬럼의 데이터 값 조사 및 카운트
within_core_counts2 = third_result4['within_core'].value_counts()

# 결과 출력
print(within_core_counts2)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-23 10:12:33.071338
within_core
0      532914327
1      504638607
NaN        69991
Name: count, dtype: int64
종료: 2024-02-23 10:13:02.253917


In [16]:
third_result4

,uid,cluster_id,pubyear2,refs_uid,refs_n,within_core
0,WOS:000325157800001,25,2013,WOS:000169875200049,59,0
1,WOS:000325157800001,25,2013,WOS:000187577400045,59,0
2,WOS:000325157800001,25,2013,WOS:000180545300005,59,0
3,WOS:000325157800001,25,2013,WOS:000188445200197,59,0
4,WOS:000325157800001,25,2013,WOS:A1972M852600031,59,0
...,...,...,...,...,...,...
1037622920,WOS:000799950200001,<NA>,<NA>,000799950200001.23,52,0
1037622921,WOS:000799950200001,<NA>,<NA>,000799950200001.22,52,0
1037622922,WOS:000799950200001,<NA>,<NA>,WOS:000316422500018,52,0
1037622923,WOS:000799950200001,<NA>,<NA>,000799950200001.48,52,0


저장하기 -> final폴더에 .

In [17]:
#오래걸릴듯...
now = datetime.datetime.now() 
print('시작:', now)

third_result4.to_csv('./mid/third/third_result4.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-23 10:16:46.721690
종료: 2024-02-23 10:52:48.718635


*< 작업 시 느낀점 >
1. 비동기 쓰레드를 사용했지만, DB 요청하는 파이썬 관련 작업만 빨라질 뿐 DB에서 데이터 불러오는 쿼리 작업은 병렬처리가 안되는거같음. -> 이 부분은 주피터 노트북 워크시트를 offset 시작점을 다르게하여 여러개 돌리는거로 추출 속도 보완함.  
2. limit 최적화 샘플링 -> 백만이나 십만이나 별차이 없어서 백만으로 함.그냥 DB에서 데이터 뽑고 로드하는게 오래걸리는거같음.  
*더 자세한건 보고서에 담음.  